In [40]:
# https://upbit.com/service_center/open_api_guide
# https://wikidocs.net/179294#api
# https://wikidocs.net/230307

In [41]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import ccxt

In [42]:
# Bithumb API로 BTC/KRW, ETH/KRW 데이터 가져오기
def get_bithumb_data(market, start_date, end_date, count=215):
    exchange = ccxt.bithumb()
    symbol = f"{market}/KRW"
    timeframe = '1d'
    since = int(pd.to_datetime(start_date).timestamp() * 1000)
    candles = exchange.fetch_ohlcv(symbol, timeframe, since, limit=count)
    df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['close']].rename(columns={'close': f"{market}_KRW"})
    return df.loc[start_date:end_date]


In [43]:
# CoinGecko API로 ETH/BTC 데이터 가져오기
def get_coingecko_data(coin, vs_currency, start_date, end_date):
    start_ts = int(pd.to_datetime(start_date).timestamp())
    end_ts = int(pd.to_datetime(end_date).timestamp())
    url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart/range"
    params = {
        'vs_currency': vs_currency,
        'from': start_ts,
        'to': end_ts
    }
    response = requests.get(url, params=params)
    data = response.json()
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
    prices.set_index('timestamp', inplace=True)
    return prices['price'].rename(name=f"{coin.upper()}_{vs_currency.upper()}")

In [44]:
# BTC Dominance (합성 데이터, CoinGecko 제한으로)
def get_btc_dominance(start_date, end_date):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    dominance = 60.86 + np.random.normal(0, 0.5, len(dates))  # 60.86% 기준
    return pd.Series(dominance, index=dates, name='BTC_Dominance')

In [45]:
# Yahoo Finance API로 WTI, USD/KRW, GOLD 데이터 가져오기
def get_yfinance_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df['Close'].rename(ticker)

In [46]:
# 데이터 수집
start_date = '2025-01-01'
end_date = '2025-07-31'
dates = pd.date_range(start=start_date, end=end_date, freq='D')

In [47]:
# Bithumb 데이터
btc_krw = get_bithumb_data('BTC', start_date, end_date)
eth_krw = get_bithumb_data('ETH', start_date, end_date)

In [48]:
# CoinGecko 데이터
eth_btc = get_coingecko_data('ethereum', 'btc', start_date, end_date)

TypeError: Series.rename() got an unexpected keyword argument 'name'

In [ ]:
# Yahoo Finance 데이터
wti = get_yfinance_data('CL=F', start_date, end_date)
usd_krw = get_yfinance_data('KRW=X', start_date, end_date)
gold = get_yfinance_data('GC=F', start_date, end_date)

C:\Users\wsm\AppData\Local\Temp\ipykernel_22092\3716775282.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


TypeError: 'str' object is not callable

In [ ]:
# Bithumb 데이터
try:
    btc_krw = get_bithumb_data('BTC', start_date, end_date)
    eth_krw = get_bithumb_data('ETH', start_date, end_date)
except Exception as e:
    print(f"Bithumb 데이터 수집 오류: {e}")
    # Fallback to synthetic data if Bithumb fails
    btc_krw = pd.Series(1.62e8 + np.cumsum(np.random.normal(0, 2e6, 215)), index=dates, name='BTC_KRW')
    eth_krw = pd.Series(5.28e6 + np.cumsum(np.random.normal(0, 8e4, 215)), index=dates, name='ETH_KRW')